# Uploading the Data via Google (BX-Book-Ratings.csv)

## Or otherwise compute on Local Host

In [1]:
from google.colab import files

uploaded = files.upload()

Saving BX-Book-Ratings.csv to BX-Book-Ratings.csv


# Libraries Used

In [0]:
import numpy as np   
import pandas as pd  

from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics.pairwise import pairwise_distances
import matplotlib.pyplot as plt
import random

# Reading of the BX-Book-Ratings.csv Dataset

In [2]:
## Reading the CSV File.

# Reading this CSV is done a liitle differently as there is ";" instead ","(Comma).
# Postprocessing to make the proper Dataframe is in the next Cell. 

df = pd.read_csv('BX-Book-Ratings.csv',sep='delimiter') # Reading the CSV File

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [0]:
## Preprocessing to form the proper Dataset.

series = df['"User-ID";"ISBN";"Book-Rating"'].astype(str)  # Converting into Series to perform the operations

df = pd.DataFrame(series.str.split(';',expand=True)) # Splitting the Series(strings) with a delimeter (';')

df.columns = ['User-ID', 'ISBN', 'Ratings'] # Changing the Column Name

In [0]:
## Removing the  '""' from each column and converting them into intergers. 

df['User-ID'] = df['User-ID'].map(lambda x: x.lstrip('"').rstrip('"'))
df['ISBN'] = df['ISBN'].map(lambda x: x.lstrip('"').rstrip('"'))
df['Ratings'] = df['Ratings'].map(lambda x: x.lstrip('"').rstrip('"'))

df['User-ID'] = df['User-ID'].astype(int)
df['Ratings'] = df['Ratings'].astype(int)
df['ISBN'] = df['ISBN'].astype(str)

In [5]:
## Outputting the Ratings Data Frame

ratings_df = df

ratings_df.head()

,User-ID,ISBN,Ratings
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [6]:
print(ratings_df.shape)

(1149780, 3)


# Filtering the Dataset to a Small Dataset

## Users with less than 200 ratings and books with less than 100 ratings are excluded

In [7]:
## Filtering out impotant books to make the dataset smaller

# users with less than 200 ratings and books with less than 100 ratings are excluded

c = ratings_df['User-ID'].value_counts()

ratings_df = ratings_df[ratings_df['User-ID'].isin(c[c >= 200].index)]

c = ratings_df['Ratings'].value_counts()

ratings_df = ratings_df[ratings_df['Ratings'].isin(c[c >= 100].index)]

ratings_df.head()

,User-ID,ISBN,Ratings
1456,277427,002542730X,10
1457,277427,0026217457,0
1458,277427,003008685X,8
1459,277427,0030615321,0
1460,277427,0060002050,0


In [8]:
print(ratings_df.shape)

(527556, 3)


# Number of Users & Books

In [9]:
num_users = ratings_df['User-ID'].unique().shape[0]
num_books = ratings_df['ISBN'].unique().shape[0]
print(str(num_users) + ' users')
print(str(num_books) + ' books')

905 users
207699 books


# Splitting of Training and Testing Dataset

In [0]:
## For Splitting the data based on percentage

def data_splitter(ratings_df, percentage):
    m,n = ratings_df.shape
    V = round(m*(100-percentage)/100)
    A = ratings_df.copy()
    
    if percentage == 100:
        return A
    
    
    a = random.sample(range(ratings_df.shape[0]),V)
    
    A['Ratings'].iloc[a] = np.nan
    
    return A

In [11]:
## Training Data (75%) of the Data

train_df = data_splitter(ratings_df, 75)


## Testing Data (25%) of the Data

test_df = data_splitter(ratings_df, 25)

print(train_df.shape)

print(test_df.shape)


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


(527556, 3)
(527556, 3)


# User Based Similarity Function

In [0]:
## Function for User-Based Recommendations

def UB_Reco(ratings_mat):
    print('The User-Based Recommendations Framework begins...')
    user_similarity = pairwise_distances(ratings_mat, metric='cosine')
    mean_user_rating = ratings_mat.mean(axis=1)
    ratings_diff = (ratings_mat - mean_user_rating[:, np.newaxis])
    pred = mean_user_rating[:, np.newaxis] + user_similarity.dot(ratings_diff) / np.array([np.abs(user_similarity).sum(axis=1)]).T
    print('The User-Based Recommendations Framework ends!')
    return pred

# MSE Calculation Function

In [0]:
## Function for MSE

def error(prediction, original):
    prediction = prediction[original.nonzero()].flatten()
    original = original[original.nonzero()].flatten()
    err = sqrt(mean_squared_error(prediction, original))
    print('Error is :'+ str(err))
    return err 

# Implementation of User-Based Recommendations Framework

In [14]:
## Conversion of DataFrame to Matrix

train_mat = train_df.pivot(index ='User-ID', columns ='ISBN', values ='Ratings').fillna(0).values

## Conversion of DataFrame to Matrix

test_mat = test_df.pivot(index ='User-ID', columns ='ISBN', values ='Ratings').fillna(0).values

print(train_mat.shape)
print(type(train_mat))
print(test_mat.shape)
print(type(test_mat))

(905, 207699)
<class 'numpy.ndarray'>
(905, 207699)
<class 'numpy.ndarray'>


In [15]:
## Performing the predictions

pred = UB_Reco(train_mat)

The User-Based Recommendations Framework begins...
The User-Based Recommendations Framework ends!


In [16]:
print('User-based Recommendation Framework RMSE: ' + str(error(pred, test_mat)))

Error is :7.953556618118948
User-based Recommendation Framework RMSE: 7.953556618118948
